## Deliverable 2. Create a Customer Travel Destinations Map.

In [22]:
# Dependencies and Setup
import pandas as pd
import requests
import numpy as np 
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Dikson,RU,73.5069,80.5464,4.03,96,84,7.02,broken clouds
1,1,Bluff,NZ,-46.6000,168.3333,57.72,71,72,14.79,broken clouds
2,2,Annau,TM,37.8876,58.5160,81.12,15,1,15.48,clear sky
3,3,Kapaa,US,22.0752,-159.3190,73.11,81,20,20.71,few clouds
4,4,Hermanus,ZA,-34.4187,19.2345,66.79,65,89,18.66,overcast clouds


In [4]:
# Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60
What is the maximum temperature you would like for your trip? 80


In [5]:
# Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.0752,-159.3190,73.11,81,20,20.71,few clouds
4,4,Hermanus,ZA,-34.4187,19.2345,66.79,65,89,18.66,overcast clouds
7,7,Cape Town,ZA,-33.9258,18.4232,76.26,52,0,5.99,clear sky
8,8,Iracoubo,GF,5.4802,-53.2011,78.04,90,100,10.54,light rain
12,12,Busselton,AU,-33.6500,115.3333,73.44,70,83,2.06,broken clouds
14,14,Saint-Philippe,RE,-21.3585,55.7679,76.59,96,100,17.11,very heavy rain
17,17,Cabo San Lucas,MX,22.8909,-109.9124,64.51,57,5,1.99,clear sky
18,18,Opuwo,NaN,-18.0607,13.8400,71.35,76,81,10.00,light rain
19,19,Ahipara,NZ,-35.1667,173.1667,63.46,90,66,2.06,broken clouds
22,22,Katherine,AU,-14.4667,132.2667,75.52,53,52,0.00,broken clouds


In [6]:
# Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                226
City                   226
Country                221
Lat                    226
Lng                    226
Max Temp               226
Humidity               226
Cloudiness             226
Wind Speed             226
Current Description    226
dtype: int64

In [8]:
# Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [9]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,73.11,few clouds,22.0752,-159.3190,
4,Hermanus,ZA,66.79,overcast clouds,-34.4187,19.2345,
7,Cape Town,ZA,76.26,clear sky,-33.9258,18.4232,
8,Iracoubo,GF,78.04,light rain,5.4802,-53.2011,
12,Busselton,AU,73.44,broken clouds,-33.6500,115.3333,
14,Saint-Philippe,RE,76.59,very heavy rain,-21.3585,55.7679,
17,Cabo San Lucas,MX,64.51,clear sky,22.8909,-109.9124,
19,Ahipara,NZ,63.46,broken clouds,-35.1667,173.1667,
22,Katherine,AU,75.52,broken clouds,-14.4667,132.2667,
25,Rikitea,PF,74.70,few clouds,-23.1203,-134.9692,


In [10]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Nearby Search API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [23]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()

In [25]:
# Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))